In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
from splinter import Browser
import requests
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Path to chromedriver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\rak_t\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


In [3]:
#Visit the Mars NASA news site
# Scrape NASA Mars News site for news title and paragraph text
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8959/nasas-insight-mars-lander-gets-a-power-boost/" target="_self">NASA's InSight Mars Lander Gets a Power Boost</a></div>

In [6]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's InSight Mars Lander Gets a Power Boost"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'The spacecraft successfully cleared some dust off its solar panels, helping to raise its energy and delay when it will need to switch off its science instruments.'

## JPL Space Images Featured Image

In [9]:
# Scrape URL featured space images. Ues splinter to find current feature Marks full-size image and assign to variable
jpl_url='https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
jpl_image_url='image/featured/mars1.jpg'
browser.visit(jpl_url)

In [10]:
# Find and click the full image button

full_image_elem = browser.find_by_css('.btn')
full_image_elem.click()

In [11]:
# Parse the resulting html with soup
html = browser.html
soup=soup(html,"html.parser")
#retrieve image url
image_url = soup.find_all('article')

In [17]:
# find the relative image url
image_path = soup.find_all('img')[2]["src"]
#image_url=image_url.split("'")[1]

In [19]:
# Use the base url to create an absolute url
#featured_image_url+image_url
featured_image_url=jpl_url+jpl_image_url
featured_image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.htmlimage/featured/mars1.jpg'

## Mars Facts

In [20]:
df = pd.read_html('http://space-facts.com/mars/')[0]


In [21]:
df.columns=['Description', 'Mars']
df.set_index('Description', inplace=True)
df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [22]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

## Hemispheres

In [ ]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [ ]:
hemisphere_image_urls = []

# First, get a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")

# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    browser.back()

In [ ]:
hemisphere_image_urls

In [ ]:
browser.quit()